# Dask DataFrames

<img src="images/dask-dataframe.svg" 
     align="right"
     width="20%"
     alt="Dask dataframes are blocked Pandas dataframes">
     
Dask Dataframes coordinate many Pandas dataframes, partitioned along an index.  They support a large subset of the Pandas API.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:57967 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 1.86 GiB


## Create Random Dataframe

We create a random timeseries of data with the following attributes:

1.  It stores a record for every 10 seconds of the year 2000
2.  It splits that year by month, keeping every month as a separate Pandas dataframe
3.  Along with a datetime index it has columns for names, ids, and numeric values

This is a small dataset of about 240 MB. Increase the number of days or reduce the frequency to practice with a larger dataset.

In [2]:
import dask
import dask.dataframe as dd
df = dask.datasets.timeseries()

Unlike Pandas, Dask DataFrames are lazy and so no data is printed here.

In [3]:
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


But the column names and dtypes are known.

In [ ]:
df.dtypes

Some operations will automatically display the data.

In [ ]:
import pandas as pd
pd.options.display.precision = 2
pd.options.display.max_rows = 10

In [4]:
df.head(3)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1028,Jerry,0.075306,0.456903
2000-01-01 00:00:01,1030,Xavier,-0.198016,-0.063594
2000-01-01 00:00:02,902,Wendy,-0.105745,-0.548232


## Use Standard Pandas Operations

Most common Pandas operations operate identically on Dask dataframes

In [ ]:
df2 = df[df.y > 0]
df3 = df2.groupby('name').x.std()
df3

Call `.compute()` when you want your result as a Pandas dataframe.

If you started `Client()` above then you may want to watch the status page during computation.

In [ ]:
computed_df = df3.compute()
type(computed_df)

In [ ]:
computed_df

## Persist data in memory

If you have the available RAM for your dataset then you can persist data in memory.  

This allows future computations to be much faster.

In [ ]:
df = df.persist()

## Time Series Operations

Because we have a datetime index time-series operations work efficiently

In [ ]:
%matplotlib inline

In [ ]:
df[['x', 'y']].resample('1h').mean().head()

In [ ]:
df[['x', 'y']].resample('24h').mean().compute().plot()

In [ ]:
df[['x', 'y']].rolling(window='24h').mean().head()

Random access is cheap along the index, but must still be computed.

In [ ]:
df.loc['2000-01-05']

In [ ]:
%time df.loc['2000-01-05'].compute()

## Set Index

Data is sorted by the index column.  This allows for faster access, joins, groupby-apply operations, etc..  However sorting data can be costly to do in parallel, so setting the index is both important to do, but only infrequently.

In [ ]:
df = df.set_index('name')
df

Because computing this dataset is expensive and we can fit it in our available RAM, we persist the dataset to memory.

In [ ]:
df = df.persist()

Dask now knows where all data lives, indexed cleanly by name.  As a result oerations like random access are cheap and efficient

In [ ]:
%time df.loc['Alice'].compute()

## Groupby Apply with Scikit-Learn

Now that our data is sorted by name we can easily do operations like random access on name, or groupby-apply with custom functions.

Here we train a different Scikit-Learn linear regression model on each name.

In [ ]:
from sklearn.linear_model import LinearRegression

def train(partition):
    est = LinearRegression()
    est.fit(partition[['x']].values, partition.y.values)
    return est

In [ ]:
df.groupby('name').apply(train, meta=object).compute()

## Further Reading

For a more in-depth introduction to Dask dataframes, see the [dask tutorial](https://github.com/dask/dask-tutorial), notebooks 04 and 07.